In [ ]:
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from mayavi import mlab
import multiprocessing


def plot_kde_3D_mayavi():

    def calc_kde(data):
        return kde(data.T)

    mu, sigma = 0, 0.1 
    x = 10*np.random.normal(mu, sigma, 5000)
    y = 10*np.random.normal(mu, sigma, 5000)
    z = 10*np.random.normal(mu, sigma, 5000)

    xyz = np.vstack([x,y,z])
    kde = gaussian_kde(xyz)

    # Evaluate kde on a grid
    xmin, ymin, zmin = x.min(), y.min(), z.min()
    xmax, ymax, zmax = x.max(), y.max(), z.max()
    xi, yi, zi = np.mgrid[xmin:xmax:30j, ymin:ymax:30j, zmin:zmax:30j]
    coords = np.vstack([item.ravel() for item in [xi, yi, zi]]) 

    # Multiprocessing
    cores = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=cores)
    results = pool.map(calc_kde, np.array_split(coords.T, 2))
    density = np.concatenate(results).reshape(xi.shape)

    # Plot scatter with mayavi
    figure = mlab.figure('DensityPlot')

    grid = mlab.pipeline.scalar_field(xi, yi, zi, density)
    min_val = density.min()
    max_val = density.max()
    mlab.pipeline.volume(
        grid, vmin=min_val, vmax=min_val + .5*(max_val-min_val)
    )

    mlab.axes()
    mlab.show()

plot_kde_3D_mayavi()